In [2]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Bidirectional , LSTM, Reshape, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

In [3]:
root_path = "/home/dadubovs/projects/deep_query_optimization/"

import sys
sys.path.insert(0, root_path)

In [4]:
import os

## LOAD DATABASE META

In [5]:
from dqo.db.models import Database

In [6]:
schema_file = os.path.join(root_path, 'dqo/data/meta/imdb_slow/schema.json')

In [7]:
db = Database.load(schema_file)

## LOAD DATA

In [8]:
from dqo.estimator import dataset

In [9]:
input_path = os.path.join(root_path, 'dqo/data/input/sql/slow/')
ds = dataset.QueriesDataset(input_path)
ds.load()
len(ds.df)

Reading data from /home/dadubovs/projects/deep_query_optimization/dqo/data/input/sql/slow/slow_query_log.csv [1/4]
loaded 11494 rows from /home/dadubovs/projects/deep_query_optimization/dqo/data/input/sql/slow/slow_query_log.csv [1/4]
Reading data from /home/dadubovs/projects/deep_query_optimization/dqo/data/input/sql/slow/slow_query_log.csv.1 [2/4]
loaded 13425 rows from /home/dadubovs/projects/deep_query_optimization/dqo/data/input/sql/slow/slow_query_log.csv.1 [2/4]
Reading data from /home/dadubovs/projects/deep_query_optimization/dqo/data/input/sql/slow/slow_query_log.csv.2 [3/4]
loaded 13453 rows from /home/dadubovs/projects/deep_query_optimization/dqo/data/input/sql/slow/slow_query_log.csv.2 [3/4]
Reading data from /home/dadubovs/projects/deep_query_optimization/dqo/data/input/sql/slow/slow_query_log.csv.3 [4/4]
loaded 13486 rows from /home/dadubovs/projects/deep_query_optimization/dqo/data/input/sql/slow/slow_query_log.csv.3 [4/4]


51858

## ENCODE QUERY

In [10]:
from dqo.estimator.neo import encoder

In [11]:
q = ds.df['query'][:3][0]

In [12]:
q

"SELECT MIN(t3.info_type_id),MIN(t5.note)  FROM comp_cast_type AS t1, movie_companies AS t2, movie_info AS t3, movie_info_idx AS t4, person_info AS t5  WHERE t1.id > 1 AND t1.kind LIKE '%fdh%' AND t2.company_id < 183436 AND t2.note LIKE '%zon%' AND t5.id < 358541 AND t5.info LIKE '%ssb%' AND t5.info_type_id = 22 AND t4.id < 1231839 AND t4.id < 1067260 AND t4.id > 982673 AND t4.info_type_id = 99 AND t4.info_type_id = 100 AND t4.note LIKE '%clu%' AND t4.note LIKE '%xor%' AND t4.note LIKE '%ttm%' AND t3.id = t4.movie_id AND t3.info = t5.info AND t3.note = t1.kind AND t4.info_type_id = t2.id"

In [13]:
encoded = ds.df['query'].apply(lambda q: encoder.encode_query(db, q))

In [14]:
ds.df['encoded'] = encoded.apply(np.array)

## PAD 

In [15]:
import tensorflow as tf

In [16]:
ds.df['input'] = ds.df['encoded'].apply(lambda v: v.reshape((-1,318)))

## SPLIT TEST / TRAIN

In [17]:
import tempfile
temp_path = os.path.join(tempfile.gettempdir(), 'dqo', 'estimator', 'neo')
temp_encoded_path = os.path.join(temp_path, 'encoded')

In [18]:
input_path_cache_key = os.path.basename(f'{input_path}')
encoded_temp_path = os.path.join(temp_encoded_path, input_path_cache_key)

basename = os.path.splitext(os.path.basename(list(ds.loaded.keys())[0]))[0]
train_file_name = f'{encoded_temp_path}{basename}_train.csv'
test_file_name = f'{encoded_temp_path}{basename}_test.csv'

In [19]:
test_split = 0.2
data_and_test_split = int(len(ds.df) * (1 - test_split))

train_data = ds.df[:data_and_test_split]
test_data = ds.df[data_and_test_split:]

In [20]:
from pathlib import Path
Path(encoded_temp_path).mkdir(parents=True, exist_ok=True)

train_data.to_csv(train_file_name, index=False, columns=['input','runtime'], mode="w+")
print(f'Saved temp data file {train_file_name} with {len(train_data)} records')

test_data.to_csv(test_file_name, index=False, columns=['input','runtime'],mode="w+")
print(f'Saved temp test file {test_file_name} with {len(test_data)} records')

Saved temp data file /tmp/dqo/estimator/neo/encoded/slow_query_log_train.csv with 41486 records
Saved temp test file /tmp/dqo/estimator/neo/encoded/slow_query_log_test.csv with 10372 records


## SCALE LABEL

In [21]:
from sklearn.preprocessing import StandardScaler

Y_train_scaler = StandardScaler()
Y_train = Y_train_scaler.fit_transform(train_data.runtime.values.reshape(-1, 1))

Y_test_scaler = StandardScaler()
Y_test = Y_test_scaler.fit_transform(test_data.runtime.values.reshape(-1, 1))

## EXAMPLE DATA


In [22]:
ds.df['input'][0:1][0].shape

(1, 318)

## Build Model

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Bidirectional , LSTM, Reshape, GRU, Embedding, Masking
from tensorflow.keras.callbacks import EarlyStopping

In [24]:
model = Sequential()
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
model = model

In [25]:
callbacks = [EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)]

## PREP X DATA

In [26]:
X_train = train_data.input.values
X_test = test_data.input.values

In [27]:
X_train.shape, Y_test.shape

((41486,), (10372, 1))

In [28]:
X_train[0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## TRAIN

In [29]:
loss = []
val_loss = []

In [1]:
history = model.fit(
    X_train,
    Y_train,
    epochs=50,
    verbose=1,
    validation_split=0.20,
    callbacks=callbacks
)

NameError: name 'model' is not defined

In [ ]:
loss += history.history['loss']
val_loss += history.history['val_loss']

In [ ]:
# Plot training & validation loss values
plt.plot(loss)
plt.plot(val_loss)
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
X_train.shape

In [ ]:
predicted = Y_train_scaler.inverse_transform(model.predict(X_train))
predictions = []
actual = []
diff = []

for i in range(len(predicted)):
    pred = predicted[i]
    act = train_data.runtime.values[i]
    predictions.append(pred)
    actual.append(act)
    diff.append(np.abs(pred - act))
from scipy import stats
stats.describe(diff)

In [ ]:
# Plot training & validation loss values
plt.plot(actual, 'blue')
plt.plot(predictions, 'red',alpha=0.5)
plt.title('Actual / Predicted')
plt.ylabel('Score')
plt.xlabel('Example')
plt.legend(['Actual', 'Predicted'], loc='upper left')
plt.show()

In [ ]:
predicted_test = Y_test_scaler.inverse_transform(model.predict(X_test))
predictions_test = []
actual_test = []
diff_test = []

for i in range(len(predicted_test)):
    pred = predicted_test[i]
    act = test_data.runtime.values[i]
    predictions_test.append(pred)
    actual_test.append(act)
    diff_test.append(np.abs(pred - act))
from scipy import stats
stats.describe(diff_test)

In [ ]:
# Plot training & validation loss values
plt.plot(actual_test, 'blue')
plt.plot(predictions_test, 'red',alpha=0.5)
plt.title('Actual / Predicted')
plt.ylabel('Score')
plt.xlabel('Example')
plt.legend(['Actual', 'Predicted'], loc='upper left')
plt.show()